In [ ]:
# Plot wind vectors and temperature in all depths in a Lon-Lat-Depth coordinate system
# author: Pia Goecke pia.goecke@uni-oldenburg.de
# 06.12.2024


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
import os
from matplotlib.colors import Normalize
from matplotlib import cm
import cmocean 

%matplotlib qt



In [2]:
#read data and rename cols
os.chdir('C:/Users/piago/Documents/Uni/5_Semester/Hiwi')
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE614_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file, skiprows=10)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df.rename(columns={'CTD_Temperature_SML' : 'CTD_Temperature_1',
                   'CTD_Temperature_30cm' : 'CTD_Temperature_30',
                   'CTD_Temperature_40cm' : 'CTD_Temperature_40',
                   'CTD_Temperature_50cm_[°C]' : 'CTD_Temperature_50',
                   'CTD_Temperature_60cm_[°C]': 'CTD_Temperature_60',
                   'CTD_Temperature_85cm' : 'CTD_Temperature_85',
                   'CTD_Temperature_100cm' : 'CTD_Temperature_100',
                   'RBR_Temperature_30cm' :  'RBR_Temperature_30',
                   'RBR_Temperature_40cm' :  'RBR_Temperature_40',
                   'RBR_Temperature_50cm' :  'RBR_Temperature_50',
                   'RBR_Temperature_60cm' :  'RBR_Temperature_60',
                   'RBR_Temperature_85cm' :  'RBR_Temperature_85',
                   'RBR_Temperature_100cm' :  'RBR_Temperature_100'
                   }, inplace= True)

df['Date'] = df.index.date
df_backup = df
# extract dfs
df_2 = df.loc['2023-03-02']
df_2 = df_2[(df_2.index < '2023-03-02 09:48') | (df_2.index >= '2023-03-02 09:49')]

df_3 = df.loc['2023-03-03']

df_8 = df.loc['2023-03-08']
df_8 = df_8[(df_8.index < '2023-03-08 09:19:10') | (df_8.index >= '2023-03-08 09:21:40')]


df_9 = df.loc['2023-03-09']
df_9.drop(df_9[df_9['CTD_Temperature_1']>6.2].index, inplace = True)


df_10 = df.loc['2023-03-10']

df_11 = df.loc['2023-03-11']

df_17 = df.loc['2023-03-17']
df_17 = df_17[df_17.index > '2023-03-17 08:03']

df_18 = df.loc['2023-03-18']
df_18 = df_18[df_18.index > '2023-03-18 07:50']

df_19 = df.loc['2023-03-19']
df_19 = df_19[(df_19.index < '2023-03-08 10:15:30') | (df_19.index >= '2023-03-19 10:16:20')]

df_20 = df.loc['2023-03-20']


all_df = [df_2, df_3, df_8, df_9, df_10, df_11, df_17, df_18, df_19, df_20]
#Detrend CTD, RBR and Campbell Air temp where reasonable

for df in [df_2, df_9, df_17]:
    # time in total seconds
    df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

    # fit polynomial with numpy.polyfit:  quadratic
    coefficients = np.polyfit(df['time_numeric'], df['CTD_Temperature_1'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_SML'] = polynomial(df['time_numeric']) #values
    df[f'detrend_2_Temp_SML'] = df['CTD_Temperature_1'] - df[f'fitted_2_Temp_SML']
    df[f'detrend_2_Temp_SML'] = df[f'detrend_2_Temp_SML'] + df[f'fitted_2_Temp_SML'].median()

    # same for all depths
    depths = [30, 40, 50, 60, 85, 100]

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"Column CTD_Temperature_{d}cm_[°C] contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()


    # ... and same for airtemp
    coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
    df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
    df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

    # ... and for RBRs
    depths = [30, 40, 50, 60, 85, 100]
    for d in depths:
        if df[f'RBR_Temperature_{d}'].dropna().empty:
            print(f"Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
        else: 
            coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

    print(f'mission {str(df['Date'][0])} detrended.')
df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]


Column CTD_Temperature_40cm_[°C] contains only NaN values. Creating nan-column for depth 40.
mission 2023-03-02 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-09 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-17 detrended.


In [85]:
df = df_9_2#df = df_3[df_3.index<'2023-03-03 13:00:00'] 
 # df = df_3[df_3.index<'2023-03-03 13:00:00'] 


In [86]:
# create wind-df. set scaling factor (0.01, 0.03, ...)

scaling_factor = 0.03 # rescale wind arrows
df_wind = df.iloc[::5] # plot every 5th wind vector

# Calculate the angle in radians for the wind direction (clockwise from North)
df_wind['angle_rad'] = np.radians(90 - df_wind['Campbell_Wind_Direction_True'])

# Calculate the u (East-West) and v (North-South) components of the wind
df_wind['u'] = df_wind['Wind_Speed_U10'] * np.sin(df_wind['angle_rad']) *scaling_factor
df_wind['v'] = df_wind['Wind_Speed_U10'] * np.cos(df_wind['angle_rad']) *scaling_factor
df_wind['w'] = 0  

# Normalize Latitude and Longitude. otherwise, arrows look weird
df_wind['Longitude_normalized'] = (df_wind['Longitude'] - df_wind['Longitude'].min()) / (df_wind['Longitude'].max() - df_wind['Longitude'].min())
df_wind['Latitude_normalized'] = (df_wind['Latitude'] - df_wind['Latitude'].min()) / (df_wind['Latitude'].max() - df_wind['Latitude'].min())
df_wind['z'] = -30


norm = Normalize(vmin=df_wind['Wind_Speed_U10'].min(), vmax=df_wind['Wind_Speed_U10'].max()) # for color 
cmap = cm.viridis



In [87]:
# create temp-df

sensor = "CTD"  # set to "CTD" or "RBR"

if sensor == "RBR":
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended RBR data is used')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'rbr_detrend_2_Temp_30', 'rbr_detrend_2_Temp_40','rbr_detrend_2_Temp_50',  'rbr_detrend_2_Temp_60', 
                 'rbr_detrend_2_Temp_85', 'rbr_detrend_2_Temp_100', 'Air_detrend_2_Temp']]

        df_temp.rename(columns={
                        'rbr_detrend_2_Temp_30' : 'RBR_Temperature_30',
                        'rbr_detrend_2_Temp_40' : 'RBR_Temperature_40',
                        'rbr_detrend_2_Temp_50' : 'RBR_Temperature_50',
                        'rbr_detrend_2_Temp_60': 'RBR_Temperature_60',
                        'rbr_detrend_2_Temp_85' : 'RBR_Temperature_85',
                        'rbr_detrend_2_Temp_100' : 'RBR_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original RBR data is used.')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'RBR_Temperature_30', 
                'RBR_Temperature_40', 
                'RBR_Temperature_50',  
                'RBR_Temperature_60', 'RBR_Temperature_85', 'RBR_Temperature_100',
                'Campbell_Air_Temperature'
                ]]
        df_temp.rename(columns={
                        'Campbell_Air_Temperature' : 'Air_Temperature_999'}, inplace= True)

    # Melt the DataFrame: Reshape from wide to long format
    df_melted = df.melt(id_vars=['Latitude', 'Longitude'], 
                        value_vars=['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50',  
                                    'RBR_Temperature_60', 'RBR_Temperature_85', 'RBR_Temperature_100', 
                                    # 'Air_Temperature_999' 
                                    ], 
                        var_name='Depth', 
                        value_name='Temperature')

    # Extract the depth information from the column names
    df_melted['Depth'] = (df_melted['Depth'].str.extract(r'(\d+)$').astype(float))
    # Now df_melted has columns: Latitude, Longitude, Temperature, and Depth
    df_temp = df_melted

elif sensor == 'CTD':
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended CTD data is used')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'detrend_2_Temp_SML', 'detrend_2_Temp_30', 'detrend_2_Temp_40',
                 'detrend_2_Temp_50',  'detrend_2_Temp_60', 'detrend_2_Temp_85', 'detrend_2_Temp_100', 'Air_detrend_2_Temp' ]]

        df.rename(columns={'detrend_2_Temp_SML' : 'CTD_Temperature_1',
                        'detrend_2_Temp_30' : 'CTD_Temperature_30',
                        'detrend_2_Temp_40' : 'CTD_Temperature_40',
                        'detrend_2_Temp_50' : 'CTD_Temperature_50',
                        'detrend_2_Temp_60': 'CTD_Temperature_60',
                        'detrend_2_Temp_85' : 'CTD_Temperature_85',
                        'detrend_2_Temp_100' : 'CTD_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original CTD data is used.')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'CTD_Temperature_1', 'CTD_Temperature_30', 
                'CTD_Temperature_40', 
                'CTD_Temperature_50',  
                'CTD_Temperature_60', 'CTD_Temperature_85', 'CTD_Temperature_100',
                'Campbell_Air_Temperature'
                ]]
        df_temp.rename(columns={
                        'Campbell_Air_Temperature' : 'Air_Temperature_999'}, inplace= True)


    # Melt the DataFrame: Reshape from wide to long format
    df_melted = df.melt(id_vars=['Latitude', 'Longitude'], 
                        value_vars=['CTD_Temperature_1', 'CTD_Temperature_30','CTD_Temperature_40',
                                    'CTD_Temperature_50',
                                    'CTD_Temperature_60', 'CTD_Temperature_85', 'CTD_Temperature_100', 
                                    # 'Air_Temperature_999' 
                                    ], 
                        var_name='Depth', 
                        value_name='Temperature')

    # Extract the depth information from the column names
    df_melted['Depth'] = (df_melted['Depth'].str.extract(r'(\d+)$').astype(float))
    # Now df_melted has columns: Latitude, Longitude, Temperature, and Depth
    df_temp = df_melted

else:
    print('That did not work. Please enter valid temperature-sensor. "CTD" or "RBR"')

# set wind-heigth to -30 drop lines where Temperature is na
df_temp['Depth'] = df_temp['Depth'].replace(999, -30)
# df = df.dropna(subset = ['Temperature'])

# Extract the columns for Latitude, Longitude, Depth, and Temperature
latitudes = df_temp['Latitude']
longitudes = df_temp['Longitude']
depths = df_temp['Depth']
temperature = df_temp['Temperature']


df_temp['Longitude_normalized'] = (df_temp['Longitude'] - df_temp['Longitude'].min()) / (df_temp['Longitude'].max() - df_temp['Longitude'].min())
df_temp['Latitude_normalized'] = (df_temp['Latitude'] - df_temp['Latitude'].min()) / (df_temp['Latitude'].max() - df_temp['Latitude'].min())


original CTD data is used.


In [88]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

T = ax.scatter(df_temp['Longitude_normalized'], df_temp['Latitude_normalized'], df_temp['Depth'], c = df_temp['Temperature'], cmap =  cmocean.cm.thermal)

W = ax.quiver(df_wind['Longitude_normalized'], df_wind['Latitude_normalized'], df_wind['z'], 
          df_wind['u'], df_wind['v'], df_wind['w'], 
          color=cmap(norm(df_wind['Wind_Speed_U10'])), 
          normalize=False, linewidth = 3)

A= ax.scatter(df_wind['Longitude_normalized'], 
            df_wind['Latitude_normalized'], 
            df_wind['z'] + df_wind['w'], 
            c=cmap(norm(df_wind['Wind_Speed_U10'])), s=50, 
            marker='o', alpha = 1)

colors = ax.scatter(df_wind['Longitude_normalized'],    # not showing, onyl for color bar
            df_wind['Latitude_normalized'], 
            df_wind['z'] + df_wind['w'], 
            c=df_wind['Wind_Speed_U10'], s=50, 
            marker='')

# # add arrow heads manually
# for idx, row in df_wind.iterrows():
#     angle = row['Campbell_Wind_Direction_True']-90  # Adjusting for the direction (North is 0, East is 90)
    
#     # Create the triangle marker for each arrowhead with the angle
#     ax.scatter(row['Longitude_normalized'] + row['u'], 
#                row['Latitude_normalized'] + row['v'], 
#                row['z'] + row['w'], 
#                c=cmap(norm(row['Wind_Speed_U10'])), s=50, 
#                marker=(3, 0, angle)) 


xmin, xmax = df_temp['Longitude_normalized'].min(), df_temp['Longitude_normalized'].max()
ymin, ymax = df_temp['Latitude_normalized'].min(), df_temp['Latitude_normalized'].max()
zmin, zmax = df_temp['Depth'].max(), -50
ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

ax.set(
    xlabel='Longitude [°]',
    ylabel='Latitude [°]',
    zlabel='Depth [cm]'
)
fig.colorbar(colors, ax=ax, fraction=0.02, pad=0.1, label='Wind Speed 10 m [m/s]')

fig.colorbar(T, ax=ax, fraction=0.02, pad=0.1, label='Temperature [°C]')
plt.show()